In [3]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [45.4 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ge

**This is to remove the previosu jar file downloaded during the wget command the jar file used for the postgres SQL**

In [4]:
!rm "/content/postgresql-42.2.9.jar"
!rm "/content/postgresql-42.2.9.jar.1"
!rm "/content/postgresql-42.2.9.jar.2"

rm: cannot remove '/content/postgresql-42.2.9.jar': No such file or directory
rm: cannot remove '/content/postgresql-42.2.9.jar.1': No such file or directory
rm: cannot remove '/content/postgresql-42.2.9.jar.2': No such file or directory


**This is wget to get the jar file and download in content directory**

In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-15 18:47:09--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.12MB/s    in 0.2s    

2021-02-15 18:47:09 (4.12 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



**Py Spark initializing and configuring the postgres jar file for startup**

In [ ]:
# Start Spark session
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("sparkDates").getOrCreate()

In [6]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

**Loading the file from Amazon website as GZ format and uncompresses it for processing**

In [7]:
# Load in data
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy/MM/dd HH:mm:ss")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [8]:
  df = df.dropna()
  df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

**The Review table updated, data extracted from main data frame for review id, customer id and will be used to update the DB table**

In [12]:
reviews_info = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"]).limit(100)
reviews_info.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



**The DB initialization, this is Heroku Postgres DB instance, DB created for BigData, there are limitations 10000 records per table**

In [13]:
mode="append"

#rds_connection_string = "postgres:admin@localhost:5432/final_project_stocks"
#jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
jdbc_url = "jdbc:postgresql://ec2-52-3-4-232.compute-1.amazonaws.com:5432/ddv6vu8jpdbjns"
config = {"user":"slcslzlanikhqj",
          "password": "b106eda2173b6ce2c35f34626fb87eafcfd0f52e9ce55f36d776827fef375f71",
          "driver":"org.postgresql.Driver"}
        

**The Review data updated on the main review_id_table in Heroku DB instance**

In [14]:
reviews_info.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

T**he Product data from the main dataframe is extracted and limited to 1000 to udpate the product table**

In [17]:
product_info = df.select(["product_id", "product_title"]).limit(1000).distinct()
product_info.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00YL0EKWE|LG G4 Case Hard T...|
|B00XK95RPQ|Selfie Stick Fibl...|
|B00SXRXUKO|Tribe AB40 Water ...|
|B009V5X1CE|RAVPower® Element...|
|B00D93OVF0|Fosmon Micro USB ...|
+----------+--------------------+
only showing top 5 rows



**The product data updated in products table in Heroku**

In [22]:
product_info.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

**The Customer data and its count based on product they chosen and grouped by to get the number of products customer reviewed**

In [20]:
#date_df = df.groupBy('date').agg({"date": "count"})

customer_info = df.groupBy(["customer_id"]).agg({"customer_id": "count"}).limit(1000)
customer_info.show(5)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   46909180|                 6|
|   42560427|                 7|
|   43789873|                 3|
|   22037526|                 2|
|   34220092|                 2|
+-----------+------------------+
only showing top 5 rows



In [21]:
customer_info.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)